In [3]:
# pip install sagemaker -U -i https://pypi.douban.com/simple

In [4]:
# pip install -r requirements.txt -U -i https://pypi.douban.com/simple

In [5]:
config = dict()

config["S3_MODEL"] = "s3://hugging-face/llm/chatglm2-6b-int4.tar.gz"
config['MODEL_NAME'] = "chatglm2-6b-int4"

In [6]:
import boto3  
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker
from time import gmtime, strftime

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

huggingface_model = HuggingFaceModel(
    model_data=config["S3_MODEL"],
    role=role,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39',
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.4xlarge',
    endpoint_name=config['MODEL_NAME'].upper() + strftime("-%Y%m%d-%H%M%S", gmtime()),
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
-----------!

In [11]:
predictor

In [9]:
from sagemaker.huggingface.model import HuggingFacePredictor

predictor = HuggingFacePredictor(
    endpoint_name='CHATGLM2-6B-INT4-20231204-030044'
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [10]:
# send request
response = predictor.predict({"input": "七月份去京都，有什么推荐的景点或者活动吗？"})
print(response)

京都七月份是夏季，很热，但是有很多美丽的景点和活动。以下是一些我推荐的建议：

1. 枫叶：京都的枫叶非常美丽，特别是元打大社和哲学之道。你可以在那里欣赏到枫叶和美丽的景色。

2. 祭典和烟火：京都的夏季有很多庆祝活动，其中最著名的是元七省大社的祭典和熊野屋久保的烟火。

3. 水上活动：京都的河流和湖泊非常适合在夏季游泳或者进行水上运动。你可以在金閣寺附近的鸭川游泳或者在滋贺郡西野湖附近的船只需坐船游览。

4. 晚期梅雨季：七月底到八月初是日本的晚期梅雨季，此时京都的街道上会变成水道，很美丽。

5. 京都塔：如果你想欣赏京都的景色，建议去京都塔，这是一个高耸的建筑物，可以展望到城市的美景。

6. 神社和寺庙：京都有很多著名的神社和寺庙，如清水寺、金阁寺和银阁寺等，这些都是值得一游的景点。

7. 美食：京都的美食也是一大亮点，你可以在京都的传统市场或者街边小摊位上品尝到当地的美食和特色小吃。

希望这些建议对你有帮助，祝你旅途愉快！


---------

**ChatGPT:** 如果您在七月份去京都，以下是一些推荐的景点和活动：

1. 平等院：位于宇治市的平等院是一座著名的佛教寺庙，以其美丽的阿弥陀殿（净土堂）而闻名。七月份时，您可以欣赏到寺庙周围的绿叶和花朵，这是一幅宜人的景象。

2. 高台寺：这座古老的佛教寺庙位于东山地区，以其美丽的苔藓庭院而著名。七月份时，苔藓更加繁茂翠绿，为寺庙增添了一份神秘的氛围。

3. 京都祇园祭：每年的7月1日至7月31日期间，京都举行着著名的祇园祭，是日本三大祭典之一。期间有各种庆祝活动，包括花车巡游、传统舞蹈表演和灯笼展示等。这是京都一年中最具盛名的节日之一。

4. 花灯路（Hanatouro）：在京都的嵐山地区，七月份会举行花灯路活动，将街道和景点点缀成美丽的灯光景观。您可以在夜晚欣赏到灯笼、花灯和投影等独特的艺术装置，给整个地区增添了浪漫和神秘的氛围。

5. 京都御苑：这是一座美丽的皇家花园，以其壮丽的景观和古老的建筑而闻名。七月份时，您可以在花园中欣赏到各种季节性的花卉，如莲花和睡莲。这里还有许多小径和池塘，适合漫步和放松。

这些只是京都众多景点和活动中的一部分。请根据您的兴趣和时间作出选择，并确保在您的行程中留出时间来探索这座美丽的古城。

In [7]:
response = predictor.predict({"input": "how to do a skip-level one-on-one meeting?"})
print(response)

Organizing a skip-level one-on-one meeting involves a few key steps. Here's a step-by-step guide:

1. Determine the purpose: Before organizing the meeting, it's important to clarify the purpose of the meeting. Are you trying to solve a specific problem or discuss a new idea? By clarifying the purpose, you can ensure that everyone involved is on the same page.
2. Identify the participants: Identify the individuals or teams who you want to include in the meeting. It's important to have a diverse group of participants that can provide different perspectives and ideas.
3. Choose a date and time: Choose a date and time that is convenient for all participants. If the meeting is taking place at a fixed location, make sure to check if everyone can attend in person or if it's too far for some.
4. Prepare materials: Prepare any necessary materials or documents that will be discussed during the meeting. This could include reports, charts, or data points.
5. Set the agenda: Set clear expectations 

------

**ChatGPT: ** A skip-level one-on-one meeting is a meeting between a manager and an employee who is not their direct report but is at a lower level within the organization. This type of meeting provides an opportunity for the manager to connect directly with employees further down the hierarchy to gain insights, provide guidance, and address any concerns or issues. Here's a step-by-step guide on how to conduct a skip-level one-on-one meeting effectively:

1. Schedule the meeting: Reach out to the employee and request a meeting, ensuring to provide a clear purpose and agenda for the discussion. Find a suitable time and location that works for both of you.

2. Set the agenda: Prepare a structured agenda for the meeting. This may include topics such as the employee's current projects, their career aspirations, challenges they're facing, feedback on the team or management, and any other relevant subjects. Share the agenda with the employee before the meeting so they can come prepared.

3. Create a comfortable environment: Foster an open and relaxed atmosphere during the meeting. Ensure privacy, choose a neutral location if possible, and make the employee feel comfortable sharing their thoughts and concerns.

4. Active listening: Be an attentive listener throughout the conversation. Encourage the employee to express their thoughts, ideas, and challenges openly. Ask open-ended questions to promote discussion and gain a deeper understanding of their perspective.

5. Provide feedback and guidance: Offer constructive feedback based on the employee's performance, strengths, and areas for improvement. Share specific examples to support your observations and suggestions. Provide guidance on their career development, potential growth opportunities, and actionable steps they can take to enhance their skills.

6. Address concerns: Use this meeting as an opportunity to identify any issues or concerns the employee may have. Listen carefully to their feedback about the team dynamics, work environment, or any challenges they're experiencing. Take their concerns seriously and discuss possible solutions or steps to address them.

7. Follow-up and action plan: Summarize the key points discussed during the meeting and mutually agree on action steps or next measures. Ensure clarity on responsibilities and deadlines for both the employee and yourself. Document the agreed-upon action plan and share it with the employee for reference.

8. Maintain confidentiality: Respect the confidentiality of the conversation. Assure the employee that their feedback will be handled appropriately and that their opinions are valued.

9. Regular follow-up: After the skip-level meeting, maintain regular communication with the employee to check on their progress, offer support, and address any ongoing issues or challenges. This helps to build trust and demonstrate your commitment to their growth and well-being.

Remember, skip-level meetings are not a substitute for regular team meetings or one-on-one meetings with direct reports. They should complement your existing management practices and contribute to a culture of open communication and employee engagement within your organization.